## Data Preprocessing

In [1]:
### Load Libraries
import pandas as pd
import numpy as np

In [46]:
# load merged dataset
file_path = "merged_dataset.csv" 
merged_df = pd.read_csv(file_path)

In [47]:
# Diplay data info
print("Initial Dataset Overview:")
print(merged_df.info())
print("\nMissing Values per Column:")
print(merged_df.isnull().sum())

Initial Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 26 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Organisation Name                              326 non-null    object 
 1   Effective Date                                 326 non-null    object 
 2   Property ID                                    326 non-null    int64  
 3   Property Type                                  326 non-null    object 
 4   Property Name                                  326 non-null    object 
 5   Property Name/Address (Where no UPRN)          317 non-null    object 
 6   Street Number                                  93 non-null     float64
 7   Street                                         220 non-null    object 
 8   Town / Post Town                               220 non-null    object 
 9   Postcode                    

#### 1. Handling Missing Values

In [48]:
# Fill missing numerical values with median
num_cols = ['Building Size - GIA (M2)', 'New Price']
for col in num_cols:
    if merged_df[col].notna().sum() > 0:  # Check if column has non-null values
        median_value = merged_df[col].median()
        merged_df[col] = merged_df[col].fillna(median_value)  # Avoid inplace=True
        print(f"Filled missing values in {col} with median: {median_value}")
    else:
        print(f"Skipping {col} as it has no non-null values to calculate the median.")

Filled missing values in Building Size - GIA (M2) with median: 95.0
Skipping New Price as it has no non-null values to calculate the median.


In [49]:
# Fill missing categorical values with 'Unknown'
cat_cols = ['Tenure Type', 'Holding Type', 'Property Type']
for col in cat_cols:
    merged_df[col] = merged_df[col].fillna('Unknown')  # Avoid inplace=True
    print(f"Filled missing values in {col} with 'Unknown'")

Filled missing values in Tenure Type with 'Unknown'
Filled missing values in Holding Type with 'Unknown'
Filled missing values in Property Type with 'Unknown'


#### 2. Data Cleaning & Formatting

In [50]:
# Convert 'Effective Date' to datetime format
merged_df['Effective Date'] = pd.to_datetime(merged_df['Effective Date'], errors='coerce')
print(f"Converted 'Effective Date' to datetime format. {merged_df['Effective Date'].isna().sum()} invalid entries were coerced to NaT.")

Converted 'Effective Date' to datetime format. 25 invalid entries were coerced to NaT.


In [51]:
# Standardize text formatting for categorical columns
cat_cols = ['Tenure Type', 'Holding Type', 'Property Type']
for col in cat_cols:
    merged_df[col] = merged_df[col].str.title().str.strip()
    print(f"Standardized text formatting for '{col}'. Unique values after formatting: {merged_df[col].nunique()}")

Standardized text formatting for 'Tenure Type'. Unique values after formatting: 7
Standardized text formatting for 'Holding Type'. Unique values after formatting: 6
Standardized text formatting for 'Property Type'. Unique values after formatting: 29


In [52]:
# Remove duplicate entries based on 'Property ID'
initial_rows = merged_df.shape[0]
merged_df.drop_duplicates(subset=['Property ID'], inplace=True)
removed_rows = initial_rows - merged_df.shape[0]
print(f"Removed {removed_rows} duplicate entries based on 'Property ID'. New total row count: {merged_df.shape[0]}.")

Removed 0 duplicate entries based on 'Property ID'. New total row count: 326.


#### 3. Feature Engineering

In [53]:
# Extract Year from 'Effective Date'
merged_df['Effective Year'] = merged_df['Effective Date'].dt.year
print(f"Extracted 'Effective Year' from 'Effective Date'. Number of unique years: {merged_df['Effective Year'].nunique()}")

Extracted 'Effective Year' from 'Effective Date'. Number of unique years: 2


In [54]:
# Calculate 'Price Per Sq Meter' (Handle Zero-Division)
merged_df['Price Per Sq Meter'] = merged_df['Base Price'] / merged_df['Building Size - GIA (M2)']
merged_df['Price Per Sq Meter'] = merged_df['Price Per Sq Meter'].replace([np.inf, -np.inf], np.nan)
merged_df['Price Per Sq Meter'] = merged_df['Price Per Sq Meter'].fillna(0)

# Output the number of zero or missing values replaced
print(f"Calculated 'Price Per Sq Meter'. Number of zero or missing values replaced: {merged_df['Price Per Sq Meter'].isna().sum()}")


Calculated 'Price Per Sq Meter'. Number of zero or missing values replaced: 0


In [55]:
print(merged_df.columns)

Index(['Organisation Name', 'Effective Date', 'Property ID', 'Property Type',
       'Property Name', 'Property Name/Address (Where no UPRN)',
       'Street Number', 'Street', 'Town / Post Town', 'Postcode', 'Ward',
       'Geo X (Easting)', 'Geo Y (Northing)', 'Tenure Type', 'Vacant',
       'Holding Type', 'Building Size - GIA (M2)', 'Site Area (Hectares)',
       'Occupied by Council / Direct Service Property',
       'Purpose / Asset Category', 'EPC Rating', 'Council Tax', 'Key Features',
       'Property Rent/Sale History', 'Base Price', 'New Price',
       'Effective Year', 'Price Per Sq Meter'],
      dtype='object')


In [56]:
# Create 'Property History Count' based on past records
merged_df['Property History Count'] = merged_df['Property Rent/Sale History'].apply(lambda x: len(str(x).split(',')) if pd.notna(x) else 0)
print(f"Created 'Property History Count' feature. Number of properties with no history: {merged_df['Property History Count'].eq(0).sum()}")

Created 'Property History Count' feature. Number of properties with no history: 0


In [57]:
# Categorize EPC Ratings into 'High', 'Medium', 'Low'
epc_mapping = {'A': 'High', 'B': 'High', 'C': 'Medium', 'D': 'Medium', 'E': 'Low', 'F': 'Low', 'G': 'Low'}
merged_df['EPC Category'] = merged_df['EPC Rating'].map(epc_mapping).fillna('Unknown')
print(f"Categorized EPC Ratings into 'High', 'Medium', 'Low'. Number of 'Unknown' EPC Ratings: {merged_df['EPC Category'].eq('Unknown').sum()}")

Categorized EPC Ratings into 'High', 'Medium', 'Low'. Number of 'Unknown' EPC Ratings: 187


#### 4. Save Cleaned Dataset

In [58]:
cleaned_file_path = "cleaned_dataset.csv"
merged_df.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved successfully at {cleaned_file_path}")

# Display final dataset info
print("\nFinal Dataset Overview:")
print(merged_df.info())

Cleaned dataset saved successfully at cleaned_dataset.csv

Final Dataset Overview:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 326 entries, 0 to 325
Data columns (total 30 columns):
 #   Column                                         Non-Null Count  Dtype         
---  ------                                         --------------  -----         
 0   Organisation Name                              326 non-null    object        
 1   Effective Date                                 301 non-null    datetime64[ns]
 2   Property ID                                    326 non-null    int64         
 3   Property Type                                  326 non-null    object        
 4   Property Name                                  326 non-null    object        
 5   Property Name/Address (Where no UPRN)          317 non-null    object        
 6   Street Number                                  93 non-null     float64       
 7   Street                                         220 non-null 

In [2]:
display_cleaned_df = pd.read_csv('cleaned_dataset.csv')
display_cleaned_df.head(10)

,Organisation Name,Effective Date,Property ID,Property Type,Property Name,Property Name/Address (Where no UPRN),Street Number,Street,Town / Post Town,Postcode,...,EPC Rating,Council Tax,Key Features,Property Rent/Sale History,Base Price,New Price,Effective Year,Price Per Sq Meter,Property History Count,EPC Category
0,"Bristol, City of",2022-07-01,938,House/Houses,St Pius X RC VA Primary School House,St Pius X RC VA Primary School House Gatehouse...,NaN,Gatehouse Avenue,Bristol,BS13 9AB,...,NaN,NaN,NaN,"2013, 2016, 2021, 2023, 2025",NaN,NaN,2022.0,0.00000,5,Unknown
1,"Bristol, City of",2022-07-01,941,Land-Unused Land,Former Fulford Centre,Former Fulford Centre Fulford Road Hartcliffe ...,NaN,NaN,NaN,BS13,...,E,1800.0,NaN,"2014, 2015, 2016, 2017, 2021",291200.0,NaN,2022.0,0.00000,5,Low
2,"Bristol, City of",2022-07-01,949,House/Houses,St Bernadettes RC VA Primary School House,St Bernadettes RC VA Primary School House 17 G...,17.0,Gladstone Road,Bristol,BS14 9LP,...,NaN,NaN,NaN,"2010, 2014, 2016, 2020, 2021, 2024",NaN,NaN,2022.0,0.00000,6,Unknown
3,"Bristol, City of",2022-07-01,950,House Service Occupancy,St Bernadettes RC SA Secondary School House,St Bernadettes RC SA Secondary School House Fo...,NaN,Fossedale Avenue,Bristol,BS14 9LS,...,NaN,NaN,NaN,"2013, 2016, 2018, 2020, 2021, 2022",NaN,NaN,2022.0,0.00000,6,Unknown
4,"Bristol, City of",2022-07-01,139020,Land-Residential,Fulford Road - Land Rear Of 95-105,Fulford Road - Land Rear Of 95-105 Fulford Roa...,NaN,NaN,NaN,BS13,...,NaN,NaN,NaN,"2016, 2017, 2021, 2022, 2024",NaN,NaN,2022.0,0.00000,5,Unknown
5,"Bristol, City of",2022-07-01,973,House/Houses,Waycroft Academy House,Waycroft Academy House Selden Road Stockwood B...,NaN,Selden Road,Bristol,BS14 8PS,...,E,1800.0,NaN,"2011, 2012, 2013, 2016, 2017, 2021, 2024",291200.0,NaN,2022.0,3462.54459,7,Low
6,"Bristol, City of",2022-07-01,985,House/Houses,Four Acres Primary School House,Four Acres Primary School House 93 Four Acres ...,93.0,Four Acres,Bristol,BS13 8RB,...,NaN,NaN,NaN,"2012, 2013, 2014, 2016, 2021, 2025",NaN,NaN,2022.0,0.00000,6,Unknown
7,"Bristol, City of",2022-07-01,1000,House Service Occupancy,Teyfant Primary School (Former) - School House,Teyfant Primary School (Former) - School House...,NaN,Teyfant Road,Bristol,BS13 0RF,...,NaN,NaN,NaN,"2014, 2015, 2016, 2019, 2021, 2022",NaN,NaN,2022.0,0.00000,6,Unknown
8,"Bristol, City of",2022-07-01,1358,Land-Verge,Wedmore Vale - Land Fronting 32,Wedmore Vale - Land Fronting 32 Wedmore Vale B...,32.0,Wedmore Vale,Bristol,BS3 5HX,...,NaN,NaN,NaN,"2014, 2016, 2021, 2024",NaN,NaN,2022.0,0.00000,4,Unknown
9,"Bristol, City of",2022-07-01,1261,Land-Verge,Dorset Road - Land Adj 15,Dorset Road - Land Adj 15 Dorset Road Henleaze...,15.0,Dorset Road,Bristol,BS9 4BJ,...,NaN,NaN,NaN,"2010, 2013, 2016, 2020, 2021",NaN,NaN,2022.0,0.00000,5,Unknown
